## Projeto Automação Web - Busca do Melhor de Preço

### DESCRIÇÃO
- Utilizar a biblioteca Selenium para buscar analisar os preços de produtos especificos e retornar o melhor site para efetuarmos a compra com preço promocional. 

### ETAPAS DO PROCESSO
- Importar a planilha com os links dos produtos
- Percorrer a planilha
	- Para cada linha:
		- Pegar o preço do site Amazon
		- Pegar o preço do site Americanas
		- Pegar o preço do site Magazine Luiza
		- Registar o local do menor preço
		- Caso tenha +20% de desconto, eviar um e-mail de aviso
- Salvar a planilha

### Passo 1 - Importar as bibliotecas

In [1]:
# Importar as bibliotecas necessárias:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import yagmail
import win32com.client as win32
from tqdm import tqdm

### Passo 2 - Importar a base de dados com os produtos

In [2]:
# Importando a base de dados:
produtos = pd.read_excel(r'Produtos.xlsx')
produtos = produtos.fillna('-')
display(produtos)

,Unnamed: 0,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/Apple-MHDA3BR-A-iPho...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,3419,3419,Original
1,1,Samsung Smart TV Crystal UHD 4K 43BU8000 2022,https://www.amazon.com.br/Samsung-Smart-Crysta...,https://www.americanas.com.br/produto/51401469...,https://www.magazineluiza.com.br/samsung-smart...,2900,2699,Amazon


### Passo 3 - Definir uma função para tratar textos e transformá-los em números decimáis

In [3]:
def transformar_texto(texto):
    return float(texto.replace('\n', ',').replace('R$', '').replace('.', '').replace(',', '.'))

### Passo 4 - Acessar os sites e armazenar os preços em variáveis

In [4]:
# Abrindo o navegador de forma "escondida":
driver = webdriver.Chrome()
driver.set_window_position(-10000,0)

# Definindo variavel com desconto mínimo que quer ser avisado por e-mail: 
desconto_min = 0.2

# Criando um loop que irá acessar os sites e análisar os preços:
enviar_email = False
for i, linha in tqdm(produtos.iterrows()):
    # Entrar no site da amazon e armazenar o preço em uma variável:
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element(By.ID, "corePrice_feature_div").text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        print('Produto {} não disponível na Amazon'.format(linha['Link Produto']))
        preco_amazon = linha['Preço Original'] * 3
    
    # Entrar no site da americanas e armazenar o preço em uma variável:
    driver.get(linha['Lojas Americanas'])
    try:
        preco_americanas = driver.find_element(By.CLASS_NAME, 'styles__PriceText-sc-x06r9i-0').text
        preco_americanas = transformar_texto(preco_americanas)
    except:
        print('Produto {} não disponível na Lojas Americanas'.format(linha['Link Produto']))
        preco_americanas = linha['Preço Original'] * 3
    
    # Entrar no site da magazine luiza e armazenar o preço em uma variável:
    driver.get(linha['Magazine Luiza'])
    try:
        preco_magazineluiza = driver.find_element(By.CLASS_NAME, 'sc-kDTinF.zuoFI.sc-dcgwPl.bvdLco').text
        preco_magazineluiza = transformar_texto(preco_magazineluiza)
    except:
        print('Produto {} não disponível na Magazine Luiza'.format(linha['Link Produto']))
        preco_magazineluiza = linha['Preço Original'] * 3
    
    
    driver.close()
    driver = webdriver.Chrome()
    
    preco_original = linha['Preço Original']
    
    lista_precos = [(preco_amazon, 'Amazon'), (preco_americanas, 'Lojas Americanas'), (preco_magazineluiza, 'Magazine Luiza'), (preco_original, 'Original')]
    
    lista_precos.sort()
    
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos.loc[i, 'Local'] = lista_precos[0][1] 
    
    if lista_precos[0][0] <= preco_original*(1-desconto_min):
        enviar_email = True  

driver.close()
print('Produtos análisados')

2it [00:44, 22.13s/it]

Produtos análisados


### Parte 5 - Salvar o arquivo atualizado e enviar o e-mail caso atenda ao críterio de desconto mínimo

In [5]:
# Salvar o arquivo
produtos.to_excel('Produtos.xlsx')

# Filtrar tabela de produtos
tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_min), :]

# Enviar e-mail:
## Definindo variáveis para envio de e-mail automatizado: 
email_destinatario = 'digite o e-mail do destinatario aqui'
email_destinatario_name = 'digite o nome do destinatario aqui'
email_subject = f'Produto(s) encontrado(s) com mais de {desconto_min:.0%} de desconto!'
email_bory = f'<p>Esses são os produtos com mais de {desconto_min:.0%} de desconto</p>{tabela_filtrada.to_html()}'

# Enviar e-mail

if enviar_email:
#     ################# Enviando com GMAIL #################
#     # Definindo as credenciais de acesso("usuário" e "senha app") ao gmail: 
#     usuario = yagmail.SMTP(user='digite seu email aqui', password='digite sua Senha APP aqui')

#     # Enviando o e-mail:
#     usuario.send(to=email_destinatario,
#                  subject=email_subject,
#                  contents=email_bory)

    ################# Enviando com Outlook #################
    # Definindo as variaveis do outlook:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    nome = email_destinatario_name
    mail.To = email_destinatario
    mail.Subject = email_subject
    mail.HTMLBody = email_bory

    # Enviando o e-mail: 
    mail.Send()

print('E-mail enviado.')

E-mail enviado.
